In [ ]:
import pandas as pd
import string
import os
import numpy as np
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
import urllib
import requests
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import roc_curve, auc, classification_report
from sklearn.preprocessing import label_binarize
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
import time
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import operator
import folium
from itertools import cycle, islice
from pandas import options
import warnings
import pickle
import re
import nltk
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
from matplotlib.pyplot import figure
from nltk.corpus import stopwords 
import nltk


nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

#Preprocessing

##Load dataframes

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = "/content/drive/My Drive/"

#Read data from csv file 
df = pd.read_csv(path + 'vaccine_train_set.csv',nrows=15976) 
df = df.dropna()
df

In [ ]:
path = "/content/drive/My Drive/"

#Read data from csv file 
df2 = pd.read_csv(path + 'vaccine_validation_set.csv',nrows=2282) 
df2 = df2.dropna()
df2

In [ ]:
df2 = df2.drop(['Unnamed: 0'], axis=1)
df2

In [ ]:
df = df.drop(['Unnamed: 0'], axis=1)
df

##Remove unneccesary characters for train and validation set

In [ ]:
df['tweet'] = df['tweet'].str.lower()
#Define function for removing special characters
def remove_special_characters(tweet, remove_digits=True):
    tweet=re.sub(r'[^a-zA-z0-9\s]','',tweet)
    return tweet
df['tweet'] = df['tweet'].apply(lambda x:''.join([i for i in x if i not in string.punctuation]))
#Apply function on review column
df['tweet']=df['tweet'].apply(remove_special_characters)


df

In [ ]:
df2['tweet'] = df2['tweet'].str.lower()
#Define function for removing special characters
def remove_special_characters(tweet, remove_digits=True):
    tweet=re.sub(r'[^a-zA-z0-9\s]','',tweet)
    return tweet
df2['tweet'] = df2['tweet'].apply(lambda x:''.join([i for i in x if i not in string.punctuation]))
#Apply function on review column
df2['tweet']=df2['tweet'].apply(remove_special_characters)


df2

In [ ]:
df['label'].value_counts()

In [ ]:
df2['label'].value_counts()

##Remove links

In [ ]:
import re
#Remove tags and links 
tag = re.compile(r'<[^>]+>')

df['tweet'] = df['tweet'].apply(lambda x: tag.sub('', x)) #removing html labels

df['tweet'] = df['tweet'].replace(r'http\S+', '', regex=True).replace(r'www.\S+', '', regex=True).replace(r'http\S+', '', regex=True).replace(r'"', '', regex=True)

df

In [ ]:
df2['tweet'] = df2['tweet'].apply(lambda x: tag.sub('', x)) #removing html labels

df2['tweet'] = df2['tweet'].replace(r'http\S+', '', regex=True).replace(r'www.\S+', '', regex=True).replace(r'http\S+', '', regex=True).replace(r'"', '', regex=True)


##Lemmatizer

In [ ]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return " ".join([lemmatizer.lemmatize(w, pos="v") for w in w_tokenizer.tokenize(text)])

df['tweet'] = df.tweet.apply(lemmatize_text).copy()

df

In [ ]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return " ".join([lemmatizer.lemmatize(w, pos="v") for w in w_tokenizer.tokenize(text)])

df2['tweet'] = df2.tweet.apply(lemmatize_text).copy()

df2

## Create test set 
####Splitted from train set

In [ ]:
x = pd.DataFrame(df, columns = ['tweet']) 
y = pd.DataFrame(df, columns = ['label']) 

# Split dataset to train and test set.
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.15, random_state=42)

X_val = pd.DataFrame(df2, columns = ['tweet']) 
Y_val = pd.DataFrame(df2, columns = ['label']) 

print("Shape of x_train: ", X_train.shape)
print("Shape of y_train: ", Y_train.shape)
print("Shape of x_test:  ", X_test.shape)
print("Shape of y_test:  ", Y_test.shape)
print("Shape of x_val:  ", X_val.shape)
print("Shape of y_val:  ", Y_val.shape)

Save the dataframes to files

In [ ]:
x_train_file = open('/content/drive/My Drive/X_train.pkl', 'wb')
pickle.dump(X_train,x_train_file)
x_train_file.close()

x_test_file = open('/content/drive/My Drive/X_test.pkl', 'wb')
pickle.dump(X_test,x_test_file)
x_test_file.close()

y_train_file = open('/content/drive/My Drive/Y_train.pkl', 'wb')
pickle.dump(Y_train,y_train_file)
y_train_file.close()

y_test_file = open('/content/drive/My Drive/Y_test.pkl', 'wb')
pickle.dump(Y_test,y_test_file)
y_test_file.close()

x_val_file = open('/content/drive/My Drive/X_val.pkl', 'wb')
pickle.dump(X_val,x_val_file)
x_val_file.close()

y_val_file = open('/content/drive/My Drive/Y_val.pkl', 'wb')
pickle.dump(Y_val,y_val_file)
y_val_file.close()